In [1]:
from tweepy import API
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import tweepy
import json
import sys
import pandas as pd

In [ ]:
#create authentication for API access

CONSUMER_KEY = '######################' #your key & token
CONSUMER_SECRET = '######################'
ACCESS_TOKEN = '######################'
ACCESS_TOKEN_SECRET = '######################'

auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = API(auth, wait_on_rate_limit=True,
          wait_on_rate_limit_notify=True)

#define Streamer

class Listener(StreamListener):
    def __init__(self):
        super(Listener,self).__init__()
        self.output_list = [ ]
        
    def on_status(self, status):
        self.output_list.append(status)
        if (not status.retweeted) and ('RT @' not in status.text):  #exclude retweet
            self.output_list.append(status)
        else:
            pass
    
    def on_error(self, status_code):
        print(status_code)
        return False

In [ ]:
#execute data collection

l = Listener()
stream = Stream(auth=api.auth, listener=l)
try:
    print('Start streaming.')
    stream.filter(track = ['###']  #your keywords or hashtags here
                  ,languages=['en']) 
    
except KeyboardInterrupt:
    print("Stopped.")
    
finally:
    print('Done.')
    stream.disconnect()

In [ ]:
#we got json
l.output_list[0]._json.keys()

In [ ]:
#convert json to dataframe

#choose the objects you need
dict_ = {'screen_name': [], 'id': [], 'location': [], 'created_at': [], 'text': [], 
        'description': []}

for i in range(0,len(l.output_list)):
    dict_['screen_name'].append(l.output_list[i]._json['user']['screen_name'])
    dict_['id'].append(str(l.output_list[i]._json['user']['id_str']))
    dict_['location'].append(l.output_list[i]._json['user']['location'])
    dict_['created_at'].append(l.output_list[i]._json['created_at'])
    dict_['text'].append(l.output_list[i]._json['text'])
    dict_['description'].append(l.output_list[i]._json['user']['description'])
    

df = pd.DataFrame(dict_)
df.sort_values(by='created_at', inplace=True, ascending=False)
df.reset_index(drop= True, inplace= True)
df.head(5)